# Datasets

## NFL Draft Data

### Source
The NFL draft data includes combine results for 4,277 players, as well as a target field which indicates if the player was selected in the first round of the NFL draft.  This data was collected from two sources:

1. Combine result data for all players in NFL combine from 2005-2017 was taken from:
2. Data for all players selected in the first round of NFL draft was taken from: http://www.nfl.com/draft/history/fulldraft
 
### Data Pre-processing
#### Compiling Data Sources
First, data from the two sources was joined using SQL to match based on player name, school, and year.  Any player who was in the first round data set was assigned a first_round_pick value of 1.  All other players were given a first_round_pick value of 0.  It was assumed that players who appeared in first round data set but not in combine results did not participate in combine.  (Note: These players were first manually scanned to make sure they were ommitted in sql due to name mispelling, use of nickname, etc.)

#### Missing Values
Many players were missing scores for certain combine events.  In these cases, a the player was given the average score for the event, position, and year.  If no player had competed in the event for the same year and position, the player was given a score of 0.  This was necessary because some events did not apply to all positions, for example in some years no Long Snapper competed in the 3 Cone drill.

#### Categorical Values
Two categorical variables were considered for the machine learning algorithms; Position and College.  Since there is no natural order or hierarchy of position, one hot encoding was used to create a new attribute for each postion.  The College attribute was treated differently - instead of using one hot encoding, a new variable POWER_FIVE was created.  This variable was set to 1 if a player attended a school in a "Power Five" NCAA football conference (ACC, Big 12, Big Ten, Pac-12, SEC).  If a player did not play in one of these conferences, the variable was set to 0.

# Decision Tree

The first algorithm used to model the two datasets was a decision tree.  Before beginning to create models, learning curves were plotted for each dataset: